# Hidden Markov Model

A Hidden Markov Model (HMM) is a statistical model that describes a system where we observe a sequence of outputs, but the underlying states that generate these outputs are hidden from us - we model both the **transitions between hidden states** and the **probabilities of observing outputs from each state**.

# Graphical Summary

![Fig](./graphical_summary/slides/HMM_placeholder.png)

# Key Formula

An HMM is defined by three key components:

**1. State Transition Matrix** $\mathbf{A}$:

$$
a_{ij} = P(z_t = j \mid z_{t-1} = i)
$$

where $z_t$ is the hidden state at time/position $t$.

**2. Emission Probability** $\mathbf{B}$:

$$
b_j(x) = P(x_t = x \mid z_t = j)
$$

where $x_t$ is the observed value at time/position $t$.

**3. Initial State Distribution** $\boldsymbol{\pi}$:

$$
\pi_i = P(z_1 = i)
$$

The joint probability of observing a sequence $\mathbf{x} = (x_1, x_2, \ldots, x_T)$ and hidden states $\mathbf{z} = (z_1, z_2, \ldots, z_T)$ is:

$$
P(\mathbf{x}, \mathbf{z}) = \pi_{z_1} \prod_{t=2}^{T} a_{z_{t-1},z_t} \prod_{t=1}^{T} b_{z_t}(x_t)
$$

# Technical Details

## Core Concepts

Hidden Markov Models extend simple Markov chains by adding an observation layer. While we cannot directly observe the states, we can observe outputs that depend on these states. This makes HMMs powerful for modeling sequential data where the underlying process is not directly observable.

**Key assumptions:**
1. **Markov property**: The current state depends only on the previous state, not the entire history: $P(z_t \mid z_1, \ldots, z_{t-1}) = P(z_t \mid z_{t-1})$
2. **Observation independence**: Given the state, the observation is independent of all other states and observations: $P(x_t \mid z_1, \ldots, z_T, x_1, \ldots, x_{t-1}) = P(x_t \mid z_t)$
3. **Stationary**: Transition and emission probabilities don't change over time (though this can be relaxed)

## Three Fundamental Problems

### 1. Evaluation Problem (Forward Algorithm)

**Question**: Given an HMM model $\lambda = (\mathbf{A}, \mathbf{B}, \boldsymbol{\pi})$ and observation sequence $\mathbf{x}$, what is $P(\mathbf{x} \mid \lambda)$?

The forward algorithm computes this efficiently using dynamic programming:

$$
\alpha_t(i) = P(x_1, \ldots, x_t, z_t = i \mid \lambda)
$$

**Recursion**:
- Initialization: $\alpha_1(i) = \pi_i b_i(x_1)$
- Recursion: $\alpha_{t+1}(j) = b_j(x_{t+1}) \sum_{i=1}^{N} \alpha_t(i) a_{ij}$
- Termination: $P(\mathbf{x} \mid \lambda) = \sum_{i=1}^{N} \alpha_T(i)$

### 2. Decoding Problem (Viterbi Algorithm)

**Question**: Given observations $\mathbf{x}$, what is the most likely sequence of hidden states?

The Viterbi algorithm finds the optimal state path:

$$
\delta_t(i) = \max_{z_1, \ldots, z_{t-1}} P(z_1, \ldots, z_{t-1}, z_t = i, x_1, \ldots, x_t \mid \lambda)
$$

**Recursion**:
- Initialization: $\delta_1(i) = \pi_i b_i(x_1)$
- Recursion: $\delta_{t+1}(j) = b_j(x_{t+1}) \max_i [\delta_t(i) a_{ij}]$
- Backtracking: Trace back from $\arg\max_i \delta_T(i)$ to recover the optimal path

### 3. Learning Problem (Baum-Welch Algorithm)

**Question**: Given observations $\mathbf{x}$, how do we estimate the parameters $\lambda = (\mathbf{A}, \mathbf{B}, \boldsymbol{\pi})$?

The Baum-Welch algorithm (an EM algorithm) iteratively updates parameters:

**E-step**: Compute posterior probabilities using forward-backward algorithm
**M-step**: Update parameters based on expected counts

## Applications in Statistical Genetics

HMMs are widely used in genetics because DNA sequences naturally exhibit:
1. **Sequential structure**: Neighboring loci are correlated due to linkage disequilibrium
2. **Hidden states**: Ancestry, haplotype structure, or functional states are not directly observed
3. **Noisy observations**: Genotyping errors, low-coverage sequencing

**Common applications**:
- **Local ancestry inference**: Hidden states represent ancestral populations
- **Genotype imputation**: Hidden states are true haplotypes, observations are typed markers
- **Copy number variation detection**: Hidden states indicate copy number, observations are read depths
- **Chromatin state annotation**: Hidden states are functional states (promoter, enhancer, etc.)

# Related Topics

- [Linkage Disequilibrium](https://statfungen.github.io/statgen-prerequisites/linkage_disequilibrium.html)
- [Hardy-Weinberg Equilibrium](https://statfungen.github.io/statgen-prerequisites/Hardy_Weinberg_equilibrium.html)
- [Bayesian Mixture Model](https://statfungen.github.io/statgen-prerequisites/Bayesian_mixture_model.html)

# Example

## Example 1: Basic HMM - Casino Dice

Let's start with a classic example that illustrates the core concepts of HMMs. Imagine a casino that sometimes switches between a fair die and a loaded die. We can observe the outcomes (die rolls), but we cannot see which die is being used. This is a perfect scenario for an HMM.

**Hidden states**: Which die is being used (Fair or Loaded)
**Observations**: The numbers we see on the die (1-6)
**Goal**: Infer which die was likely used for each roll, given only the sequence of observed outcomes

This example parallels many genetics problems where we have:
- Hidden states: ancestry, true genotypes, functional states
- Observations: genotype data with noise/errors
- Goal: Infer the hidden states from observed data

In [ ]:
rm(list=ls())
set.seed(123)

# Define the HMM parameters for the casino problem
# Hidden states: 1 = Fair die, 2 = Loaded die

# Initial state probabilities
# Start with 95% probability of fair die
pi <- c(0.95, 0.05)
names(pi) <- c("Fair", "Loaded")

# State transition matrix
# Rows = current state, Columns = next state
# Most of the time, stay in current state (diagonal is high)
A <- matrix(c(
  0.95, 0.05,   # From Fair: 95% stay Fair, 5% switch to Loaded
  0.10, 0.90    # From Loaded: 10% switch to Fair, 90% stay Loaded
), nrow = 2, byrow = TRUE)
rownames(A) <- colnames(A) <- c("Fair", "Loaded")

# Emission probabilities
# Rows = states, Columns = possible observations (1-6)
# Fair die: uniform probability 1/6 for each outcome
# Loaded die: biased toward 6
B <- matrix(c(
  1/6, 1/6, 1/6, 1/6, 1/6, 1/6,      # Fair die
  1/10, 1/10, 1/10, 1/10, 1/10, 1/2  # Loaded die (50% chance of 6)
), nrow = 2, byrow = TRUE)
rownames(B) <- c("Fair", "Loaded")
colnames(B) <- paste0("Die=", 1:6)

cat("HMM Parameters:\n")
cat("\nInitial state probabilities:\n")
print(pi)
cat("\nState transition matrix A:\n")
print(A)
cat("\nEmission probabilities B:\n")
print(B)

### Simulate Data from the HMM

Let's generate a sequence of observations by simulating the hidden state sequence first, then generating observations based on those states.

In [ ]:
# Function to simulate from HMM
simulate_hmm <- function(pi, A, B, T) {
  n_states <- length(pi)
  n_obs <- ncol(B)
  
  # Storage
  states <- integer(T)
  observations <- integer(T)
  
  # Initial state
  states[1] <- sample(1:n_states, size = 1, prob = pi)
  observations[1] <- sample(1:n_obs, size = 1, prob = B[states[1], ])
  
  # Generate sequence
  for (t in 2:T) {
    # Sample next state based on current state
    states[t] <- sample(1:n_states, size = 1, prob = A[states[t-1], ])
    # Sample observation from current state
    observations[t] <- sample(1:n_obs, size = 1, prob = B[states[t], ])
  }
  
  return(list(states = states, observations = observations))
}

# Simulate sequence of length 300
T <- 300
sim_data <- simulate_hmm(pi, A, B, T)
true_states <- sim_data$states
observations <- sim_data$observations

# Convert states to names for clarity
state_names <- c("Fair", "Loaded")

cat("\nSimulated Data (first 50 positions):\n")
cat("Observations: ", observations[1:50], "\n")
cat("True states:  ", state_names[true_states[1:50]], "\n")

# Summary statistics
cat("\nSummary Statistics:\n")
cat("Total time points:", T, "\n")
cat("Fair die used:", sum(true_states == 1), "times (", 
    round(100*sum(true_states == 1)/T, 1), "%)\n")
cat("Loaded die used:", sum(true_states == 2), "times (", 
    round(100*sum(true_states == 2)/T, 1), "%)\n")
cat("Number of state switches:", sum(diff(true_states) != 0), "\n")
cat("\nObserved frequency of 6's:", sum(observations == 6), "out of", T, 
    "(", round(100*sum(observations == 6)/T, 1), "%)\n")

### Visualize the Simulated Data

Let's plot the observations and the true hidden states to see the pattern.

In [ ]:
# Plot observations with true states colored
par(mfrow = c(2, 1), mar = c(4, 4, 3, 1))

# Plot 1: Observations colored by true state
plot(1:T, observations, pch = 16, cex = 0.8,
     col = c("steelblue", "red")[true_states],
     xlab = "Time", ylab = "Die Roll",
     main = "Observed Die Rolls (colored by true hidden state)",
     ylim = c(0.5, 6.5))
legend("topleft", legend = c("Fair die", "Loaded die"),
       col = c("steelblue", "red"), pch = 16, cex = 0.8)

# Plot 2: True hidden states over time
plot(1:T, true_states, type = "s", lwd = 2, col = "darkgreen",
     xlab = "Time", ylab = "State",
     main = "True Hidden States Over Time",
     ylim = c(0.5, 2.5), yaxt = "n")
axis(2, at = 1:2, labels = c("Fair", "Loaded"))

par(mfrow = c(1, 1))

### Inference: Forward Algorithm

Now let's implement the forward algorithm to compute the probability of the observed sequence. This algorithm efficiently computes $P(\mathbf{x} \mid \lambda)$ using dynamic programming.

**Key insight**: At each time step, we maintain the probability of seeing the observations up to that point and being in each possible state. This lets us avoid enumerating all possible state sequences (which would be exponential in the length).

In [ ]:
# Forward algorithm implementation
forward_algorithm <- function(observations, pi, A, B) {
  T <- length(observations)
  n_states <- length(pi)
  
  # Initialize forward variables (log scale for numerical stability)
  log_alpha <- matrix(0, nrow = T, ncol = n_states)
  
  # Initialization: t=1
  log_alpha[1, ] <- log(pi) + log(B[, observations[1]])
  
  # Recursion: t=2 to T
  for (t in 2:T) {
    for (j in 1:n_states) {
      # Sum over all possible previous states
      log_alpha[t, j] <- log(sum(exp(log_alpha[t-1, ] + log(A[, j])))) + 
                         log(B[j, observations[t]])
    }
  }
  
  # Termination: log P(observations | model)
  log_prob <- log(sum(exp(log_alpha[T, ])))
  
  return(list(log_alpha = log_alpha, log_prob = log_prob))
}

# Run forward algorithm
forward_result <- forward_algorithm(observations, pi, A, B)

cat("\nForward Algorithm Results:\n")
cat("Log probability of observed sequence: ", forward_result$log_prob, "\n")
cat("Probability of observed sequence: ", exp(forward_result$log_prob), "\n")

# Show forward probabilities at a few time points
cat("\nForward probabilities at selected time points:\n")
selected_times <- c(1, 50, 100, 150, 200, 250, 300)
for (t in selected_times) {
  alpha_t <- exp(forward_result$log_alpha[t, ])
  alpha_t <- alpha_t / sum(alpha_t)  # Normalize for interpretation
  cat(sprintf("Time %3d: P(Fair|obs[1:%d]) = %.3f, P(Loaded|obs[1:%d]) = %.3f\n", 
              t, t, alpha_t[1], t, alpha_t[2]))
}

### Inference: Viterbi Algorithm

The Viterbi algorithm finds the most likely sequence of hidden states given the observations. Instead of summing over all paths (like forward algorithm), it finds the single best path.

**Key difference from forward algorithm**: Use max instead of sum when combining paths.

In [ ]:
# Viterbi algorithm implementation
viterbi_algorithm <- function(observations, pi, A, B) {
  T <- length(observations)
  n_states <- length(pi)
  
  # Initialize
  log_delta <- matrix(0, nrow = T, ncol = n_states)
  psi <- matrix(0, nrow = T, ncol = n_states)
  
  # Initialization: t=1
  log_delta[1, ] <- log(pi) + log(B[, observations[1]])
  
  # Recursion: t=2 to T
  for (t in 2:T) {
    for (j in 1:n_states) {
      # Find the most likely previous state
      temp <- log_delta[t-1, ] + log(A[, j])
      psi[t, j] <- which.max(temp)
      log_delta[t, j] <- max(temp) + log(B[j, observations[t]])
    }
  }
  
  # Termination: find best final state
  best_path <- integer(T)
  best_path[T] <- which.max(log_delta[T, ])
  
  # Backtracking: trace back the best path
  for (t in (T-1):1) {
    best_path[t] <- psi[t+1, best_path[t+1]]
  }
  
  log_prob <- max(log_delta[T, ])
  
  return(list(best_path = best_path, log_prob = log_prob))
}

# Run Viterbi algorithm
viterbi_result <- viterbi_algorithm(observations, pi, A, B)
predicted_states <- viterbi_result$best_path

cat("\nViterbi Algorithm Results:\n")
cat("Log probability of best path: ", viterbi_result$log_prob, "\n")

# Compare predicted vs true states
accuracy <- sum(predicted_states == true_states) / T
cat("\nAccuracy: ", round(100*accuracy, 2), "%\n")

# Confusion matrix
confusion <- table(True = state_names[true_states], 
                   Predicted = state_names[predicted_states])
cat("\nConfusion Matrix:\n")
print(confusion)

# Show first 50 predictions
cat("\nFirst 50 positions:\n")
cat("Observations: ", observations[1:50], "\n")
cat("True states:  ", state_names[true_states[1:50]], "\n")
cat("Predicted:    ", state_names[predicted_states[1:50]], "\n")

### Visualize Inference Results

In [ ]:
# Plot comparing true vs predicted states
par(mfrow = c(3, 1), mar = c(3, 4, 2, 1))

# Plot 1: Observations
plot(1:T, observations, pch = 16, cex = 0.6, col = "gray40",
     xlab = "", ylab = "Die Roll",
     main = "Observed Die Rolls",
     ylim = c(0.5, 6.5))

# Plot 2: True states
plot(1:T, true_states, type = "s", lwd = 2, col = "darkgreen",
     xlab = "", ylab = "State",
     main = "True Hidden States",
     ylim = c(0.5, 2.5), yaxt = "n")
axis(2, at = 1:2, labels = c("Fair", "Loaded"))

# Plot 3: Predicted states with errors highlighted
plot(1:T, predicted_states, type = "s", lwd = 2, col = "steelblue",
     xlab = "Time", ylab = "State",
     main = "Predicted States (Viterbi)",
     ylim = c(0.5, 2.5), yaxt = "n")
axis(2, at = 1:2, labels = c("Fair", "Loaded"))

# Highlight errors
errors <- which(predicted_states != true_states)
if (length(errors) > 0) {
  points(errors, predicted_states[errors], col = "red", pch = 16, cex = 0.8)
}

par(mfrow = c(1, 1))

## Example 2: Local Ancestry Inference

Local ancestry inference (LAI) identifies which ancestral population contributed each genomic segment in admixed individuals. For example, African Americans have chromosomes that are mosaics of African and European ancestry. HMMs are ideal for this problem because:

- **Hidden states**: Ancestry at each locus (e.g., African, European)
- **Observations**: Genotypes at SNP markers
- **Sequential structure**: Neighboring SNPs tend to have the same ancestry due to linkage (recombination is rare)

This example is inspired by ELAI (Efficient Local Ancestry Inference), which uses a two-layer HMM. We'll implement a simplified version to demonstrate the core concepts.

**Key insight**: The transition probabilities in the HMM are related to the recombination rate and the number of generations since admixture. More generations → more recombination → more switches between ancestries.

In [ ]:
rm(list=ls())
set.seed(456)

# Simulate admixed individuals with two ancestral populations
# Population 1 (e.g., European), Population 2 (e.g., African)

n_individuals <- 10
n_snps <- 1000
n_pops <- 2

# Allele frequencies in ancestral populations
# Population 1: frequencies centered around 0.3
# Population 2: frequencies centered around 0.7
# Some SNPs have similar frequencies (uninformative)
# Some SNPs have very different frequencies (ancestry-informative markers)

freq_pop1 <- runif(n_snps, 0.2, 0.4)
freq_pop2 <- runif(n_snps, 0.6, 0.8)

# Make some SNPs uninformative (similar frequencies in both populations)
uninformative <- sample(1:n_snps, size = n_snps * 0.3)  # 30% uninformative
freq_pop2[uninformative] <- freq_pop1[uninformative] + runif(length(uninformative), -0.1, 0.1)
freq_pop2[freq_pop2 < 0] <- 0.05
freq_pop2[freq_pop2 > 1] <- 0.95

# Store frequencies in matrix
freq_matrix <- rbind(freq_pop1, freq_pop2)
rownames(freq_matrix) <- c("Pop1", "Pop2")

cat("Ancestral Population Allele Frequencies:\n")
cat("First 10 SNPs:\n")
print(round(freq_matrix[, 1:10], 3))

# Calculate informativeness for each SNP
delta <- abs(freq_pop1 - freq_pop2)
cat("\nAncestry-informative markers (delta > 0.3):", sum(delta > 0.3), "\n")
cat("Uninformative markers (delta < 0.1):", sum(delta < 0.1), "\n")

### Simulate Admixed Genomes

We'll simulate individuals with ~60% ancestry from Population 1 and ~40% from Population 2. The ancestry switches along the chromosome according to recombination.

In [ ]:
# HMM parameters for ancestry inference
# Assume 5 generations since admixture, genetic distance ~1 Morgan
generations <- 5
genetic_length <- 1  # Morgan

# Expected number of recombination events = generations * genetic_length
expected_switches <- generations * genetic_length

# Transition probability between SNPs
# Approximate: prob_switch = expected_switches / n_snps
prob_switch <- expected_switches / n_snps
prob_stay <- 1 - prob_switch

# Initial ancestry proportions (global ancestry)
pi_anc <- c(0.6, 0.4)  # 60% Pop1, 40% Pop2

# Transition matrix (simplified - assumes symmetric switching)
A_anc <- matrix(c(
  prob_stay, prob_switch,
  prob_switch, prob_stay
), nrow = 2, byrow = TRUE)

cat("\nLocal Ancestry HMM Parameters:\n")
cat("Generations since admixture:", generations, "\n")
cat("Expected ancestry switches:", expected_switches, "\n")
cat("Transition probability per SNP:", round(prob_switch, 4), "\n")
cat("\nTransition matrix:\n")
print(round(A_anc, 4))

# Simulate admixed genomes
simulate_admixed_genome <- function(n_snps, pi, A, freq_matrix) {
  # Simulate ancestry for each haplotype (2 per individual)
  ancestry_hap1 <- integer(n_snps)
  ancestry_hap2 <- integer(n_snps)
  
  # Haplotype 1
  ancestry_hap1[1] <- sample(1:2, 1, prob = pi)
  for (i in 2:n_snps) {
    ancestry_hap1[i] <- sample(1:2, 1, prob = A[ancestry_hap1[i-1], ])
  }
  
  # Haplotype 2 (independent)
  ancestry_hap2[1] <- sample(1:2, 1, prob = pi)
  for (i in 2:n_snps) {
    ancestry_hap2[i] <- sample(1:2, 1, prob = A[ancestry_hap2[i-1], ])
  }
  
  # Generate genotypes based on ancestry
  genotypes <- integer(n_snps)
  for (i in 1:n_snps) {
    # Allele from haplotype 1
    allele1 <- rbinom(1, 1, freq_matrix[ancestry_hap1[i], i])
    # Allele from haplotype 2
    allele2 <- rbinom(1, 1, freq_matrix[ancestry_hap2[i], i])
    genotypes[i] <- allele1 + allele2
  }
  
  return(list(genotypes = genotypes, 
              ancestry_hap1 = ancestry_hap1,
              ancestry_hap2 = ancestry_hap2))
}

# Simulate multiple individuals
sim_data_list <- lapply(1:n_individuals, function(i) {
  simulate_admixed_genome(n_snps, pi_anc, A_anc, freq_matrix)
})

# Extract genotype matrix
genotype_matrix <- sapply(sim_data_list, function(x) x$genotypes)
genotype_matrix <- t(genotype_matrix)  # individuals x SNPs

cat("\nSimulated genotype matrix:\n")
cat("Dimensions:", dim(genotype_matrix), "(individuals x SNPs)\n")

# Show ancestry for first individual
true_anc_hap1 <- sim_data_list[[1]]$ancestry_hap1
true_anc_hap2 <- sim_data_list[[1]]$ancestry_hap2

cat("\nIndividual 1 ancestry summary:\n")
cat("Haplotype 1: Pop1 =", round(100*mean(true_anc_hap1 == 1), 1), "%, Pop2 =", 
    round(100*mean(true_anc_hap1 == 2), 1), "%\n")
cat("Haplotype 2: Pop1 =", round(100*mean(true_anc_hap2 == 1), 1), "%, Pop2 =", 
    round(100*mean(true_anc_hap2 == 2), 1), "%\n")
cat("Ancestry switches (Hap1):", sum(diff(true_anc_hap1) != 0), "\n")
cat("Ancestry switches (Hap2):", sum(diff(true_anc_hap2) != 0), "\n")

### Visualize True Ancestry

Let's visualize the true local ancestry for the first individual.

In [ ]:
par(mfrow = c(2, 1), mar = c(3, 4, 2, 1))

# Plot ancestry for both haplotypes
plot(1:n_snps, true_anc_hap1, type = "s", lwd = 2, col = "darkblue",
     xlab = "", ylab = "Ancestry", ylim = c(0.5, 2.5),
     main = "Individual 1: True Local Ancestry (Haplotype 1)",
     yaxt = "n")
axis(2, at = 1:2, labels = c("Pop1", "Pop2"))

plot(1:n_snps, true_anc_hap2, type = "s", lwd = 2, col = "darkred",
     xlab = "SNP Position", ylab = "Ancestry", ylim = c(0.5, 2.5),
     main = "Individual 1: True Local Ancestry (Haplotype 2)",
     yaxt = "n")
axis(2, at = 1:2, labels = c("Pop1", "Pop2"))

par(mfrow = c(1, 1))

### Infer Local Ancestry Using HMM

Now we'll use the Viterbi algorithm to infer the ancestry at each locus. The emission probabilities are based on how likely each genotype is under each ancestry state.

**Emission probability calculation**: For a diploid genotype $g \in \{0, 1, 2\}$ at a SNP with allele frequencies $p_1$ (Pop1) and $p_2$ (Pop2), if the average ancestry is population $k$, the expected frequency is approximately $p_k$. Under Hardy-Weinberg equilibrium:

$$P(g \mid \text{ancestry} = k) = \binom{2}{g} p_k^g (1-p_k)^{2-g}$$

This is a simplification; more sophisticated methods model haplotype-level ancestry.

In [ ]:
# Function to compute emission probability for ancestry inference
# P(genotype | ancestry)
compute_emission_prob <- function(genotype, freq_pop1, freq_pop2) {
  # For each ancestry state, compute P(genotype | frequency)
  # Using binomial probability under HWE
  
  g <- genotype
  
  # Pop1
  p1 <- freq_pop1
  prob_pop1 <- dbinom(g, 2, p1)
  
  # Pop2
  p2 <- freq_pop2
  prob_pop2 <- dbinom(g, 2, p2)
  
  return(c(prob_pop1, prob_pop2))
}

# Viterbi for local ancestry inference
viterbi_ancestry <- function(genotypes, freq_pop1, freq_pop2, pi, A) {
  n_snps <- length(genotypes)
  n_states <- 2
  
  # Initialize
  log_delta <- matrix(-Inf, nrow = n_snps, ncol = n_states)
  psi <- matrix(0, nrow = n_snps, ncol = n_states)
  
  # Emission probabilities for first SNP
  emit1 <- compute_emission_prob(genotypes[1], freq_pop1[1], freq_pop2[1])
  log_delta[1, ] <- log(pi) + log(emit1 + 1e-10)  # Add small value to avoid log(0)
  
  # Recursion
  for (i in 2:n_snps) {
    emit_i <- compute_emission_prob(genotypes[i], freq_pop1[i], freq_pop2[i])
    
    for (j in 1:n_states) {
      temp <- log_delta[i-1, ] + log(A[, j] + 1e-10)
      psi[i, j] <- which.max(temp)
      log_delta[i, j] <- max(temp) + log(emit_i[j] + 1e-10)
    }
  }
  
  # Backtracking
  best_path <- integer(n_snps)
  best_path[n_snps] <- which.max(log_delta[n_snps, ])
  
  for (i in (n_snps-1):1) {
    best_path[i] <- psi[i+1, best_path[i+1]]
  }
  
  return(best_path)
}

# Infer ancestry for first individual
ind1_genotypes <- genotype_matrix[1, ]
inferred_ancestry <- viterbi_ancestry(ind1_genotypes, freq_pop1, freq_pop2, pi_anc, A_anc)

# Note: We're inferring average ancestry (not haplotype-specific)
# True average ancestry
true_avg_ancestry <- round((true_anc_hap1 + true_anc_hap2) / 2)
true_avg_ancestry[true_avg_ancestry == 2] <- 2  # If both haplotypes are Pop2
true_avg_ancestry[true_avg_ancestry == 1] <- 1  # If average is Pop1

# For comparison, let's use a simpler rule: if both haplotypes are same, that's the ancestry
# Otherwise, call it as mixed (but for this simple model, we'll just compare to one haplotype)

cat("\nLocal Ancestry Inference Results:\n")
cat("Inferred global ancestry proportions:\n")
cat("Pop1:", round(100*mean(inferred_ancestry == 1), 1), "%\n")
cat("Pop2:", round(100*mean(inferred_ancestry == 2), 1), "%\n")

# Compare with true (using haplotype 1 as reference)
accuracy_anc <- sum(inferred_ancestry == true_anc_hap1) / n_snps
cat("\nAccuracy (vs Hap1):", round(100*accuracy_anc, 2), "%\n")

# Inferred ancestry switches
cat("Inferred ancestry switches:", sum(diff(inferred_ancestry) != 0), "\n")
cat("True ancestry switches (Hap1):", sum(diff(true_anc_hap1) != 0), "\n")

### Compare Inferred vs True Ancestry

In [ ]:
par(mfrow = c(3, 1), mar = c(3, 4, 2, 1))

# Plot 1: True ancestry (Hap1)
plot(1:n_snps, true_anc_hap1, type = "s", lwd = 2, col = "darkgreen",
     xlab = "", ylab = "Ancestry", ylim = c(0.5, 2.5),
     main = "True Local Ancestry (Haplotype 1)",
     yaxt = "n")
axis(2, at = 1:2, labels = c("Pop1", "Pop2"))

# Plot 2: Inferred ancestry
plot(1:n_snps, inferred_ancestry, type = "s", lwd = 2, col = "steelblue",
     xlab = "", ylab = "Ancestry", ylim = c(0.5, 2.5),
     main = "Inferred Local Ancestry (HMM)",
     yaxt = "n")
axis(2, at = 1:2, labels = c("Pop1", "Pop2"))

# Plot 3: Ancestry informativeness (delta)
plot(1:n_snps, delta, type = "h", col = "coral", lwd = 1,
     xlab = "SNP Position", ylab = "Freq Diff",
     main = "Ancestry Informativeness (|p1 - p2|)")
abline(h = 0.3, lty = 2, col = "red", lwd = 2)
text(n_snps*0.8, 0.35, "Highly informative", col = "red")

par(mfrow = c(1, 1))

### Summary: Local Ancestry Inference

The HMM successfully identifies genomic segments from different ancestral populations. Key observations:

1. **Accuracy depends on informativeness**: Regions with highly ancestry-informative markers (large frequency differences) are easier to infer correctly.

2. **Sequential structure helps**: The HMM leverages linkage disequilibrium - neighboring SNPs tend to have the same ancestry, which improves accuracy even at uninformative markers.

3. **Transition probabilities encode recombination**: The rate of ancestry switching in the HMM reflects the number of generations since admixture and the recombination rate.

4. **Real methods are more sophisticated**: Tools like ELAI use two-layer HMMs and model haplotype-level ancestry explicitly, achieving higher accuracy than this simplified example.

## Example 3: Genotype Imputation

Genotype imputation infers missing genotypes at untyped markers using a reference panel of haplotypes. This is crucial in GWAS because:
- Genotyping arrays only assay ~500K-1M SNPs, but the genome has millions of variants
- Different studies use different arrays, making meta-analysis difficult
- Imputation allows us to test association at variants not on the array

**The HMM framework**:
- **Hidden states**: Reference panel haplotypes (each haplotype is a state)
- **Observations**: Genotypes at typed markers
- **Key idea**: Each study sample is a mosaic of reference haplotypes due to shared ancestry

We'll implement a simplified version inspired by the Li-Stephens model used in tools like IMPUTE and Minimac.

In [ ]:
rm(list=ls())
set.seed(789)

# Simulate a reference panel of haplotypes
n_ref_haps <- 50    # Number of reference haplotypes
n_markers <- 500    # Total number of markers in the region
n_typed <- 100      # Number of typed markers (20%)

# Generate reference haplotypes
# We'll create structured haplotypes with some shared segments (LD)
# Start with founders and simulate recombination

n_founders <- 10
freq <- runif(n_markers, 0.1, 0.9)  # Allele frequencies

# Create founder haplotypes
founder_haps <- matrix(rbinom(n_founders * n_markers, 1, rep(freq, each = n_founders)),
                       nrow = n_founders, ncol = n_markers)

# Generate reference haplotypes as mosaics of founders
# This creates LD structure
ref_haplotypes <- matrix(0, nrow = n_ref_haps, ncol = n_markers)

for (i in 1:n_ref_haps) {
  # Start with a random founder
  current_founder <- sample(1:n_founders, 1)
  
  for (j in 1:n_markers) {
    # Small probability of switching to another founder (recombination)
    if (runif(1) < 0.02) {
      current_founder <- sample(1:n_founders, 1)
    }
    ref_haplotypes[i, j] <- founder_haps[current_founder, j]
  }
}

cat("Reference Panel:\n")
cat("Number of haplotypes:", n_ref_haps, "\n")
cat("Number of markers:", n_markers, "\n")
cat("Reference panel dimensions:", dim(ref_haplotypes), "\n")

# Select which markers are typed vs untyped
typed_indices <- sort(sample(1:n_markers, n_typed))
untyped_indices <- setdiff(1:n_markers, typed_indices)

cat("\nTyped markers:", n_typed, "\n")
cat("Untyped markers:", length(untyped_indices), "\n")

### Create Study Sample

Now we'll create a study sample that is related to the reference panel (shares haplotype segments). We'll then "mask" genotypes at untyped markers and try to impute them.

In [ ]:
# Create a study sample haplotype as a mosaic of reference haplotypes
# This simulates the fact that the study sample shares recent ancestry with the reference

true_study_haplotype <- integer(n_markers)
true_state_path <- integer(n_markers)  # Which reference haplotype at each position

# Start with a random reference haplotype
current_ref <- sample(1:n_ref_haps, 1)
true_state_path[1] <- current_ref

for (j in 1:n_markers) {
  # Small probability of switching to another reference haplotype
  if (j > 1 && runif(1) < 0.01) {
    current_ref <- sample(1:n_ref_haps, 1)
  }
  true_state_path[j] <- current_ref
  true_study_haplotype[j] <- ref_haplotypes[current_ref, j]
}

# Add some genotyping errors at typed positions
observed_study_haplotype <- true_study_haplotype
error_rate <- 0.005
errors <- rbinom(n_typed, 1, error_rate)
error_positions <- typed_indices[errors == 1]
observed_study_haplotype[error_positions] <- 1 - observed_study_haplotype[error_positions]

# Mask untyped positions (set to NA)
observed_study_haplotype_masked <- observed_study_haplotype
observed_study_haplotype_masked[untyped_indices] <- NA

cat("\nStudy Sample:\n")
cat("True number of reference switches:", sum(diff(true_state_path) != 0), "\n")
cat("Genotyping errors introduced:", length(error_positions), "\n")
cat("Observed alleles (typed only):", sum(!is.na(observed_study_haplotype_masked)), "\n")
cat("Missing alleles (untyped):", sum(is.na(observed_study_haplotype_masked)), "\n")

### HMM for Imputation

The Li-Stephens HMM treats each reference haplotype as a hidden state. The key components:

**Transition probabilities**: Probability of switching from one reference haplotype to another
$$P(\text{switch}) \approx \exp(-\rho d)$$
where $\rho$ is recombination rate and $d$ is genetic distance.

**Emission probabilities**: At typed markers, how well does the reference haplotype match the observed genotype?
$$P(\text{obs} | \text{ref}) = \begin{cases} 1 - \epsilon & \text{if match} \\ \epsilon & \text{if mismatch} \end{cases}$$
where $\epsilon$ is the genotyping error rate.

In [ ]:
# Simplified imputation using HMM forward-backward algorithm
# We'll compute posterior probability of each allele at untyped markers

# HMM parameters
theta <- 0.01  # Switching rate between reference haplotypes
epsilon <- 0.01  # Genotyping error rate

# For simplicity, uniform initial distribution
pi_imp <- rep(1/n_ref_haps, n_ref_haps)

# Transition matrix: mostly stay in same state
# P(stay in same ref haplotype) = 1 - theta
# P(switch to any other) = theta / (n_ref_haps - 1)
prob_stay <- 1 - theta
prob_switch <- theta / (n_ref_haps - 1)

A_imp <- matrix(prob_switch, nrow = n_ref_haps, ncol = n_ref_haps)
diag(A_imp) <- prob_stay

# Forward algorithm for imputation
forward_imputation <- function(observed_hap, ref_haplotypes, pi, A, epsilon, typed_indices) {
  n_markers <- length(observed_hap)
  n_states <- nrow(ref_haplotypes)
  
  # Forward variables
  log_alpha <- matrix(-Inf, nrow = n_markers, ncol = n_states)
  
  # Only compute at typed positions to save time
  # At untyped positions, just propagate forward
  
  # Initialization at first typed marker
  first_typed <- typed_indices[1]
  obs_allele <- observed_hap[first_typed]
  
  for (k in 1:n_states) {
    ref_allele <- ref_haplotypes[k, first_typed]
    match_prob <- ifelse(obs_allele == ref_allele, 1 - epsilon, epsilon)
    log_alpha[first_typed, k] <- log(pi[k]) + log(match_prob)
  }
  
  # Forward recursion (only at typed markers for efficiency)
  for (i in 2:length(typed_indices)) {
    curr_pos <- typed_indices[i]
    prev_pos <- typed_indices[i-1]
    
    # Distance-based transition (simplified)
    distance <- curr_pos - prev_pos
    A_dist <- A
    
    obs_allele <- observed_hap[curr_pos]
    
    for (k in 1:n_states) {
      ref_allele <- ref_haplotypes[k, curr_pos]
      match_prob <- ifelse(obs_allele == ref_allele, 1 - epsilon, epsilon)
      
      # Sum over previous states
      log_alpha[curr_pos, k] <- log(sum(exp(log_alpha[prev_pos, ] + log(A_dist[, k])))) + 
                                 log(match_prob)
    }
  }
  
  return(log_alpha)
}

cat("\nRunning forward algorithm for imputation...\n")
cat("This may take a moment...\n")

log_alpha <- forward_imputation(observed_study_haplotype_masked, ref_haplotypes, 
                                pi_imp, A_imp, epsilon, typed_indices)

cat("Forward algorithm completed.\n")

### Compute Imputation Dosages

At each untyped marker, we compute the posterior probability that the allele is 1 (alternative allele). This is called the "dosage" and can be used directly in association testing.

In [ ]:
# For each untyped marker, compute P(allele = 1 | observed data)
# This requires knowing which reference haplotype we're likely copying

imputed_dosages <- rep(NA, n_markers)
imputed_dosages[typed_indices] <- observed_study_haplotype_masked[typed_indices]

# For each untyped marker
for (pos in untyped_indices) {
  # Find the nearest typed markers before and after
  typed_before <- typed_indices[typed_indices < pos]
  typed_after <- typed_indices[typed_indices > pos]
  
  if (length(typed_before) == 0 || length(typed_after) == 0) {
    # At edges, use population frequency
    imputed_dosages[pos] <- mean(ref_haplotypes[, pos])
    next
  }
  
  nearest_before <- max(typed_before)
  nearest_after <- min(typed_after)
  
  # Get posterior distribution over states at flanking positions
  alpha_before <- exp(log_alpha[nearest_before, ])
  alpha_before <- alpha_before / sum(alpha_before)
  
  # Weight each reference haplotype by its posterior probability
  # and take weighted average of alleles
  weighted_alleles <- sum(alpha_before * ref_haplotypes[, pos])
  imputed_dosages[pos] <- weighted_alleles
}

# Convert dosages to hard calls (0 or 1) for accuracy assessment
imputed_calls <- round(imputed_dosages)

# Evaluate imputation accuracy at untyped markers
accuracy_untyped <- sum(imputed_calls[untyped_indices] == true_study_haplotype[untyped_indices]) / 
                    length(untyped_indices)

cat("\nImputation Results:\n")
cat("Accuracy at untyped markers:", round(100*accuracy_untyped, 2), "%\n")

# Show some examples
cat("\nFirst 10 untyped markers:\n")
example_untyped <- untyped_indices[1:10]
results_df <- data.frame(
  Position = example_untyped,
  True_Allele = true_study_haplotype[example_untyped],
  Imputed_Dosage = round(imputed_dosages[example_untyped], 3),
  Imputed_Call = imputed_calls[example_untyped],
  Correct = imputed_calls[example_untyped] == true_study_haplotype[example_untyped]
)
print(results_df)

### Visualize Imputation Results

In [ ]:
# Focus on a smaller region for visualization
region_start <- 1
region_end <- 200
region <- region_start:region_end

typed_in_region <- typed_indices[typed_indices >= region_start & typed_indices <= region_end]
untyped_in_region <- untyped_indices[untyped_indices >= region_start & untyped_indices <= region_end]

par(mfrow = c(2, 1), mar = c(4, 4, 3, 1))

# Plot 1: True vs Imputed alleles
plot(region, true_study_haplotype[region], type = "n",
     xlab = "Marker Position", ylab = "Allele",
     main = "Genotype Imputation: True vs Imputed",
     ylim = c(-0.2, 1.2))

# True alleles
points(region, true_study_haplotype[region], pch = 16, col = "black", cex = 0.8)

# Typed positions (observed)
points(typed_in_region, observed_study_haplotype[typed_in_region], 
       pch = 1, col = "blue", cex = 1.5, lwd = 2)

# Imputed positions
points(untyped_in_region, imputed_calls[untyped_in_region], 
       pch = 4, col = "red", cex = 1)

# Highlight imputation errors
errors_in_region <- untyped_in_region[imputed_calls[untyped_in_region] != 
                                       true_study_haplotype[untyped_in_region]]
if (length(errors_in_region) > 0) {
  points(errors_in_region, imputed_calls[errors_in_region], 
         pch = 4, col = "orange", cex = 2, lwd = 3)
}

legend("topleft", 
       legend = c("True", "Typed (observed)", "Imputed correct", "Imputed error"),
       pch = c(16, 1, 4, 4),
       col = c("black", "blue", "red", "orange"),
       pt.cex = c(0.8, 1.5, 1, 2),
       pt.lwd = c(1, 2, 1, 3))

# Plot 2: Imputation dosages (uncertainty)
plot(region, imputed_dosages[region], type = "n",
     xlab = "Marker Position", ylab = "Allele Dosage",
     main = "Imputation Dosages (Posterior Probability of Alt Allele)",
     ylim = c(-0.1, 1.1))

# Show dosages at untyped positions
points(untyped_in_region, imputed_dosages[untyped_in_region],
       pch = 16, col = "purple", cex = 0.8)

# Add reference lines
abline(h = 0.5, lty = 2, col = "gray")
text(region_end - 20, 0.55, "Uncertain (dosage = 0.5)", col = "gray")

# Highlight confident vs uncertain imputation
confident <- untyped_in_region[abs(imputed_dosages[untyped_in_region] - 0.5) > 0.3]
uncertain <- untyped_in_region[abs(imputed_dosages[untyped_in_region] - 0.5) <= 0.3]

if (length(confident) > 0) {
  points(confident, imputed_dosages[confident], pch = 16, col = "darkgreen", cex = 1.2)
}
if (length(uncertain) > 0) {
  points(uncertain, imputed_dosages[uncertain], pch = 16, col = "orange", cex = 1.2)
}

par(mfrow = c(1, 1))

### Summary: Genotype Imputation

The HMM-based imputation method successfully recovers most untyped genotypes by leveraging:

1. **Shared haplotype segments**: Study samples are mosaics of reference haplotypes due to shared ancestry. The HMM identifies which reference haplotypes the study sample is copying.

2. **Linkage disequilibrium**: Typed markers provide information about nearby untyped markers because alleles are correlated.

3. **Transition probabilities**: The switching rate between reference haplotypes is based on recombination rate and time to common ancestor.

4. **Dosages vs hard calls**: Rather than making hard 0/1 calls, imputation produces dosages (posterior probabilities) that quantify uncertainty. This is important for:
   - Rare variants (often uncertain)
   - Regions with weak LD
   - Association testing (dosages can be used directly)

**Real imputation tools** (IMPUTE2, Minimac, Beagle) use more sophisticated models:
- Pre-phase haplotypes using more complex HMMs
- Use forward-backward algorithm for full posterior
- Handle large reference panels (10,000s of haplotypes) efficiently
- Model haplotype clusters to reduce computational complexity
- Achieve >98% accuracy for common variants with large reference panels

# Conclusion

Hidden Markov Models are fundamental tools in statistical genetics that elegantly handle sequential data with hidden structure. The three examples demonstrate how HMMs:

1. **Model sequential dependencies**: The Markov property captures the fact that neighboring genomic positions are correlated (due to LD, limited recombination)

2. **Handle uncertainty**: Emission probabilities account for noise (genotyping errors, measurement uncertainty)

3. **Enable efficient inference**: Dynamic programming algorithms (forward, Viterbi, Baum-Welch) make exact inference tractable even for long sequences

**Key algorithms recap**:
- **Forward**: Compute $P(\text{observations})$
- **Viterbi**: Find most likely state sequence
- **Forward-Backward**: Compute posterior state probabilities
- **Baum-Welch**: Learn HMM parameters

HMMs remain workhorses in modern genomics, underlying tools for imputation (IMPUTE2, Minimac), local ancestry (RFMix, ELAI), copy number variation (HMMcopy), and chromatin state annotation (ChromHMM).